# Deepfake Google Colab
Note:
 
Run each code cell one at a time and in sequential order. 
 
Before running the code cells, go to runtime > change runtime type > and select GPU.
 
If you want to do this again just factory reset runtime from the runtime menu.

**Download dependencies and clone deepfake github**

In [ ]:
!pip install PyYAML==5.3.1
!git clone https://github.com/AliaksandrSiarohin/first-order-model

**Change working directory to where the github repo was cloned**

In [ ]:
cd first-order-model

**Mount Google Drive**
 
You'll get a link in the following output window. Click that choose which accounts drive you want to mount. It'll show a authentication code that you can copy and paste into the output window to mount your Google drive. 

Notes: 
1. make a folder in your google drive named "first-order-motion-model"
 
2. upload the following files to that folder:
  https://drive.google.com/file/d/1xRD9rVjTizmqBnr6F4nOId5M08sCMKS9/view?usp=sharing
 
  https://drive.google.com/file/d/1RdFq0hdT-Lnr8k6iw7e4JMRzo4I5f5YN/view?usp=sharing
 
3. Make sure the image you want to deepfake is named "02" and is in png format. The filename will be "02.png"
 
4. The deepfake template should be named "04" and should be in mp4 format. The filename should be "04.mp4"
 
5. The deepfake template for dame de/ baka mitai: https://drive.google.com/file/d/1-_K5D6gr7-tte0c_9VRXETAt6MQ9iSVd/view?usp=drivesdk

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

**Load the driving video and the source image into the program**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
 
source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/02.png')
driving_video = imageio.mimread('/content/gdrive/My Drive/first-order-motion-model/04.mp4')
 
 
#Resize image and video to 256x256
 
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
 
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
 
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    
 
HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load encodings and perform image animation**

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                                          checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')
 
print('Proceed to next cell')

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

# Steps for editing generated video

You'll need to speed up the generated video 3x so that it'll sync up with whatever music. You can use kapwing or another editor for this. If you're doing this in kapwing, you'll first need to speed it up 2x and speed up the resulting video again 1.5x. This is because kapwing doesn't have a 3x speed feature. You might have to do this